<a href="https://colab.research.google.com/github/LGLV/ML_regression/blob/main/regression_problem_predicting_molecular_bioactivity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (
    RandomForestRegressor,
    GradientBoostingRegressor
)
from sklearn.neighbors import KNeighborsRegressor
import warnings
warnings.filterwarnings("ignore")

# Optional: XGBoost
try:
    from xgboost import XGBRegressor
except ImportError:
    print("⚠️ Install XGBoost with: pip install xgboost")

# 2. Load dataset
df = pd.read_csv("bioactivity.csv")
X = df.drop("bioactivity", axis=1)
y = df["bioactivity"]

# 3. Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Define regression models
models = {
    "Linear Regression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "ElasticNet": ElasticNet(),
    "SVR": SVR(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor(),
    "KNN": KNeighborsRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "XGBoost": XGBRegressor()
}

# 5. Cross-validation
print("=== Regression Model Evaluation ===")
results = {}
for name, model in models.items():
    pipeline = Pipeline([
        ("scaler", StandardScaler()),
        ("regressor", model)
    ])
    scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='r2')
    mean_score = np.mean(scores)
    results[name] = mean_score
    print(f"{name}: Mean R² = {mean_score:.3f}")

# 6. Top 3 models
top3 = sorted(results.items(), key=lambda x: x[1], reverse=True)[:3]
print("\nTop 3 models by R²:")
for name, score in top3:
    print(f"{name}: {score:.3f}")

# 7. Hyperparameter tuning
best_model_name = top3[0][0]
print(f"\nHyperparameter tuning for: {best_model_name}")

if best_model_name == "Random Forest":
    param_grid = {
        "n_estimators": [100, 200],
        "max_depth": [None, 10, 20],
        "min_samples_split": [2, 5]
    }
    model = RandomForestRegressor()
elif best_model_name == "SVR":
    param_grid = {
        "C": [0.1, 1, 10],
        "kernel": ["linear", "rbf"]
    }
    model = SVR()
elif best_model_name == "XGBoost":
    param_grid = {
        "n_estimators": [100, 200],
        "max_depth": [3, 6, 10],
        "learning_rate": [0.01, 0.1, 0.3]
    }
    model = XGBRegressor()
else:
    param_grid = {}
    model = models[best_model_name]

# 8. Fit model
if param_grid:
    gs = GridSearchCV(model, param_grid, cv=5, scoring='r2')
    gs.fit(X_train, y_train)
    final_model = gs.best_estimator_
    print("Best hyperparameters:", gs.best_params_)
else:
    final_model = model
    final_model.fit(X_train, y_train)

# 9. Final evaluation
y_pred = final_model.predict(X_test)
print("\n=== Test Set Results ===")
print(f"R²:  {r2_score(y_test, y_pred):.3f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred):.3f}")
print(f"RMSE: {mean_squared_error(y_test, y_pred, squared=False):.3f}")
